<a href="https://colab.research.google.com/github/profteachkids/CHE3023/blob/main/CoolingTower.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
from plotly.subplots import make_subplots
import plotly.io as pio
pio.templates.default='plotly_dark'
from scipy.integrate import solve_ivp
from scipy.optimize import root

In [2]:
def Pv(T):
    return 10**(8.07131- 1730.63/(233.426+(T-273.15)))/760*101325

In [9]:
h=10 #W/(m2 K)
R=8.314 #J/(mol K)
rho_air = 1.2 #kg/m3
Cair = 1000 #J/(kg K)
Cwg = 2000 #J/(kg K)
CwL = 4190 #J/(kg K)
Hv = 40700 #J/mol
kc = h/(rho_air * Cair)

a= 5 #m2/m3
Mw = 18e-3 #kg/mol
Mair = 29e-3 #kg/mol

vel = 5 #m/s
L = 0.5 #kg/(m2 s)
P = 101325 #Pa

height=20
z_plot=np.linspace(0,height,21)
TwFeed = 70 +273.15
TaFeed = 25 +273.15
yaFeed = 0.5*Pv(TaFeed)/P
Gair = P*Mair/(R*TaFeed)*vel

yExit_guess = yaFeed*1.1
TgExit_guess = TaFeed + 5
sol=None

In [10]:
def converge_feed_air_T_y(v2):
    yExit = v2[0]
    TgExit = v2[1]

    global sol
    def dyTwTg(z, v):
        y=v[0]
        Tw=v[1]
        Tg=v[2]

        Gw = y/(1-y)*Gair*Mw/Mair
        G = Gw+Gair
        M = (Gair + Gw)/(Gair/Mair + Gw/Mw)
        Cg = (Cair*Gair + Cwg*Gw)/G

        Nw = kc/R/Tg*(Pv(Tw)-y*P)
        dy= -a*M/G*Nw
        dTw = a/L/CwL*(h*(Tg-Tw) - Hv*Nw )
        dTg = a*h/G/Cg*(Tg-Tw)
        return dy, dTw, dTg

    sol=solve_ivp(dyTwTg, (0,height), (yExit, TwFeed, TgExit), method='Radau', dense_output=True).sol(z_plot)
    return sol[0,-1] - yaFeed, sol[2,-1]-TaFeed

In [11]:
root(converge_feed_air_T_y,(yExit_guess, TgExit_guess))

    fjac: array([[-0.0396639 ,  0.99921308],
       [-0.99921308, -0.0396639 ]])
     fun: array([ 5.10854466e-11, -2.11326778e-08])
 message: 'The solution converged.'
    nfev: 6
     qtf: array([ 2.21956481e-06, -9.95621072e-08])
       r: array([-27.90702121,   1.14863407,  -0.04555204])
  status: 1
 success: True
       x: array([2.51174510e-02, 3.00179723e+02])

In [18]:
fig = make_subplots(rows=3,cols=1)
fig.add_scatter(x=z_plot,y=sol[0,:], name = 'yH2O',row=1,col=1)
fig.add_scatter(x=z_plot, y=sol[1,:], name='Tw', row=2, col=1)
fig.add_scatter(x=z_plot, y=sol[2,:],name='Tg', row=3,col=1)
fig.update_layout(width=600,height=900)